# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [1]:
! pip install nltk
! pip install scikit-learn
! pip install pymorphy2
! pip install fuzzywuzzy
! pip install python-Levenshtein

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [3]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [5]:
from nltk.tokenize import word_tokenize
import pandas as pd
import nltk

descriptions = pd.read_csv('preprocessed_descriptions.csv')
df1 = ''.join(descriptions['preprocessed_descriptions'].astype(str))
unfiltered = nltk.FreqDist(word_tokenize(df1))
words = dict((word, freq) for word, freq in unfiltered.items() if not word.isdigit())

words

{'an': 2945,
 'original': 669,
 'recipe': 14158,
 'created': 256,
 'by': 2670,
 'chef': 354,
 'scott': 2,
 'meskan': 1,
 'georges': 2,
 'at': 3454,
 'the': 39495,
 'cove': 2,
 'we': 2105,
 'enjoyed': 214,
 'this': 19284,
 'when': 2731,
 'visited': 22,
 'restaurant': 489,
 'in': 13666,
 'la': 68,
 'jolla': 1,
 'california': 89,
 'is': 20239,
 'requested': 132,
 'so': 5521,
 'often': 402,
 'they': 2788,
 'have': 5499,
 'it': 18980,
 'printed': 60,
 'and': 30238,
 'ready': 296,
 'hostess': 17,
 'stand': 107,
 'its': 3699,
 'unbeatable': 2,
 'but': 6324,
 'i': 21414,
 'do': 1343,
 'a': 32404,
 'pretty': 458,
 'good': 3386,
 'job': 30,
 'home': 786,
 'too': 1252,
 'if': 3754,
 'say': 324,
 'myselfmy': 1,
 'children': 114,
 'their': 635,
 'friends': 390,
 'ask': 113,
 'for': 15772,
 'my': 7810,
 'homemade': 290,
 'popsicles': 4,
 'morning': 285,
 'noon': 3,
 'night': 488,
 'never': 696,
 'turn': 175,
 'them': 2561,
 'down': 485,
 'who': 874,
 'am': 767,
 'to': 23421,
 'tell': 151,
 'that': 6

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [6]:
import random

random_words = random.sample(list(words.keys()), 5)


In [7]:
from nltk.metrics.distance import (
 edit_distance,
 edit_distance_align,
 binary_distance,
 jaccard_distance,
 masi_distance,
 interval_distance,
 custom_distance,
 presence,
 fractional_presence,
)

# # результат при substitution_cost=1

for word1 in random_words:
    for word2 in random_words:
        print(f'расстояние между {word1} и {word2}: {edit_distance(word1, word2)}')


расстояние между daysi и daysi: 0
расстояние между daysi и peasthis: 6
расстояние между daysi и evolution: 8
расстояние между daysi и bayou: 3
расстояние между daysi и farmsmake: 7
расстояние между peasthis и daysi: 6
расстояние между peasthis и peasthis: 0
расстояние между peasthis и evolution: 8
расстояние между peasthis и bayou: 7
расстояние между peasthis и farmsmake: 8
расстояние между evolution и daysi: 8
расстояние между evolution и peasthis: 8
расстояние между evolution и evolution: 0
расстояние между evolution и bayou: 8
расстояние между evolution и farmsmake: 9
расстояние между bayou и daysi: 3
расстояние между bayou и peasthis: 7
расстояние между bayou и evolution: 8
расстояние между bayou и bayou: 0
расстояние между bayou и farmsmake: 8
расстояние между farmsmake и daysi: 7
расстояние между farmsmake и peasthis: 8
расстояние между farmsmake и evolution: 9
расстояние между farmsmake и bayou: 8
расстояние между farmsmake и farmsmake: 0


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [8]:
def find_nearest_words(word, k, words):
    nearest = []
    for token in list(words.keys()):
        nearest.append((edit_distance(word, token), token))
    nearest.sort()
    return nearest[:k]

find_nearest_words('timesyou', 5, words)

[(0, 'timesyou'),
 (1, 'timeyou'),
 (2, 'piesyou'),
 (2, 'takesyou'),
 (2, 'timesgot')]

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [9]:
from nltk.stem import SnowballStemmer
import re
import pymorphy2
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

snb_stemmer_en = SnowballStemmer('english')


word = list(words.keys())
stemmed_word = [snb_stemmer_en.stem(t) for t in word]


normalized_word = [WordNetLemmatizer().lemmatize(w) for w in word]

d = {
    'stemmed_word': pd.Series(stemmed_word, index=word),
    'normalized_word': pd.Series(normalized_word, index=word)
    }

df_res = pd.DataFrame(d)
df_res

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Никита\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,stemmed_word,normalized_word
an,an,an
original,origin,original
recipe,recip,recipe
created,creat,created
by,by,by
...,...,...
measuresthis,measuresthi,measuresthis
augsburg,augsburg,augsburg
consensus,consensus,consensus
planted,plant,planted


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [10]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Никита\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
from nltk.corpus import stopwords

all_stop_words = stopwords.words('english')
all_words = len(word)
all_finded_words = 0
new_word = []
for i in range(all_words):
    if not word[i] in all_stop_words:
        new_word.append(word[i])
    else:
        all_finded_words += 1

new_word

['original',
 'recipe',
 'created',
 'chef',
 'scott',
 'meskan',
 'georges',
 'cove',
 'enjoyed',
 'visited',
 'restaurant',
 'la',
 'jolla',
 'california',
 'requested',
 'often',
 'printed',
 'ready',
 'hostess',
 'stand',
 'unbeatable',
 'pretty',
 'good',
 'job',
 'home',
 'say',
 'myselfmy',
 'children',
 'friends',
 'ask',
 'homemade',
 'popsicles',
 'morning',
 'noon',
 'night',
 'never',
 'turn',
 'tell',
 'variety',
 'substitute',
 'different',
 'flavours',
 'frozen',
 'juice',
 'grape',
 'fruit',
 'punch',
 'tropical',
 'etcthese',
 'go',
 'surprised',
 'even',
 'memy',
 'sisterinlaw',
 'made',
 'us',
 'family',
 'get',
 'together',
 'delicious',
 'little',
 'messy',
 'make',
 'worth',
 'effort',
 'helper',
 'think',
 'fondue',
 'romantic',
 'casual',
 'dinner',
 'wonderful',
 'theatre',
 'snack',
 'served',
 'robust',
 'red',
 'wine',
 'serve',
 'rice',
 'small',
 'salad',
 'almond',
 'pilaf',
 'great',
 'accompaniment',
 'posted',
 'separately',
 'cook',
 'meat',
 'must',


In [33]:
word = sorted(word, key= lambda x: words[x], reverse=True)
new_word = sorted(new_word, key= lambda x: words[x], reverse=True)
print('Было: ' + ', '.join(word[:10]))
print('Стало: ' + ', '.join(new_word[:10]))

Было: the, a, and, to, i, is, this, it, of, for
Стало: recipe, make, use, time, great, like, easy, made, one, good


In [30]:
print(f'Процент: {all_finded_words/all_words * 100}%')

Процент: 0.2899168620763164%


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [45]:
descriptions = pd.read_csv('preprocessed_descriptions.csv')
random_nums = random.sample(range(descriptions.shape[0]), 5)
df_rnd = list(descriptions['preprocessed_descriptions'].iloc[random_nums])
df_rnd

['by stephanie jaworski',
 'a combination of a few lasagna recipes',
 'this recipe has been in our family for a long time  we use it for any type of fruit cobbler  instead of the berries you could use 1 or 2 cans of canned cherry pie filling  i just used freshly picked berries for this recipe today',
 'when i was younger my favorite restaurant in tulsa was peking garden one of my favorites menu items they had was crabmeat corn soup unfortunatly the place went out of business and i was pretty bummed years later i came across this recipe its not quite the same but its pretty darn close',
 'not really a pie at all but because its flat its called a pita in greek  heres the version i make  i like it cause its quick and can satisfy any chocoholic']

In [51]:
from sklearn.feature_extraction.text import (CountVectorizer, TfidfVectorizer)

tv = TfidfVectorizer()

corpus_tv = tv.fit_transform(df_rnd)

corpus_tv.toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.57735027, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.57735027, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [54]:
from scipy.spatial import distance

names = list(descriptions['name'].iloc[random_nums])
m = []
for desc1 in corpus_tv.toarray():
    l = []
    for desc2 in corpus_tv.toarray():
        l.append(1 - distance.cosine(desc1, desc2))
        
    m.append(l)
    
d = {name: pd.Series(m[ind], index=names) for ind, name in enumerate(names)}
res_df = pd.DataFrame(d)

res_df

,lemon ginger scones,lasagna bolognese,black raspberry cobbler,easy crabmeat and corn soup,sokolatopita greek chocolate cake
lemon ginger scones,1.0,0.000000,0.000000,0.000000,0.000000
lasagna bolognese,0.0,1.000000,0.088389,0.053207,0.000000
black raspberry cobbler,0.0,0.088389,1.000000,0.115096,0.075073
easy crabmeat and corn soup,0.0,0.053207,0.115096,1.000000,0.171355
sokolatopita greek chocolate cake,0.0,0.000000,0.075073,0.171355,1.000000


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

Рецепты, между которыми значение косинусого расстояния равно 1, являются идентичными, так как косинус угла между их векторным представлениями равен 1, следовательно угл между векторами равен 0 и они равны.
Рецепты между которыми значение косинусого расстояния равно 0, являются сильно различающимися, так как их векторные представления сильно различаются.
Из оставшихся рецептов, наиболее похожи между собой 'easy crabmeat and corn soup' и 'sokolatopita greek chocolate cake', так как их косинусовое расстояние наиболее большое и равно 0,171355